In [1]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from progressbar import ProgressBar
import random
import matplotlib.pyplot as plt
from tensorflow.keras import backend as BE

import os
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "4"
config=tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess=tf.compat.v1.Session(config=config)     

In [2]:
def get_acc(predict_label, ground_truth=None):
    for i in predict_label[0]:
        if i[0] == ground_truth:
            return True
    return False

def softmax( f ):
    # instead: first shift the values of f so that the highest number is 0:
    f -= np.max(f) # f becomes [-666, -333, 0]
    return np.exp(f) / np.sum(np.exp(f))  # safe to do, gives the correct answer

def get_APFD(Gini_indexs, ground_truth_label, predicted_confidence, top_set=None):
    o_i = 0
    pbar = ProgressBar()
    wrong_num = 0
    for i in pbar(range(0, len(Gini_indexs))):
        if top_set is not None:
            if not get_acc(predict_label=decode_predictions(predicted_confidence[Gini_indexs[i]], top=top_set), 
                           ground_truth=ground_truth_label[Gini_indexs[i]]):
                o_i = o_i+i
#                 print(i, o_i)
                wrong_num = wrong_num+1
        else:
            if np.argmax(ground_truth_label[Gini_indexs[i]]) != np.argmax(predicted_confidence[Gini_indexs[i]]):
                o_i = o_i+i
                wrong_num = wrong_num+1
    APFD = 1 - o_i/(len(Gini_indexs)*wrong_num) + 1/(2*len(Gini_indexs))
    return APFD, len(Gini_indexs), wrong_num

def get_RAUC(Gini_indexs, ground_truth_label, predicted_confidence, top_set=None):
    pre_y_axis = []
    o_i = 0
    wrong_num = 0
    pbar = ProgressBar()
    for i in pbar(range(0, len(Gini_indexs))):
        if top_set is not None:
            if not get_acc(predict_label=decode_predictions(predicted_confidence[Gini_indexs[i]], top=top_set), 
                           ground_truth=ground_truth_label[Gini_indexs[i]]):  
                o_i = o_i+1
                wrong_num = wrong_num+1
                pre_y_axis.append(o_i)
            else:
                pre_y_axis.append(o_i)
        else:
            if np.argmax(ground_truth_label[Gini_indexs[i]]) != np.argmax(predicted_confidence[Gini_indexs[i]]):
                o_i = o_i+1
                wrong_num = wrong_num+1
                pre_y_axis.append(o_i)
            else:
                pre_y_axis.append(o_i)
    true_y_axis = wrong_num*(len(Gini_indexs)-wrong_num) + (wrong_num+1)*wrong_num/2
    RAUC = np.sum(pre_y_axis)/true_y_axis
#     print("RAUC: ", RAUC)
    return RAUC, len(Gini_indexs), wrong_num

In [3]:
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
X_train = np.reshape(X_train,[-1,28*28])
X_test = np.reshape(X_test,[-1,28*28])
Y_train = to_categorical(Y_train,10)
Y_test = to_categorical(Y_test,10)

In [4]:
input_shape = 28*28

model = Sequential()
model.add(Dense(input_shape=(input_shape,), units=input_shape, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=10, activation='softmax'))
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # model training
# model.fit(X_train, Y_train, epochs=1, batch_size=512)

# MODEL_PATH = "/public/liujiawei/huawei/ZHB/ADF-master/models/"
# model.save(MODEL_PATH+"fmnist_FC4_bad.h5")

# del model

In [5]:
MODEL_PATH = "/public/liujiawei/huawei/ZHB/ADF-master/models/"
model = load_model(MODEL_PATH+"fmnist_FC4_bad.h5")

In [6]:
loss_acc = model.evaluate(X_test, Y_test, batch_size=128)
print('accuracy', loss_acc[-1])

79/79 [==============================] - 2s 2ms/step - loss: 1.2974 - accuracy: 0.7201
accuracy 0.7200999855995178


In [20]:
img_num = 200
X_test = X_test[0:img_num]
Y_test = Y_test[0:img_num]
Gini = []
predicted_confidence = []
index1 = 0
index2 = 0
pbar = ProgressBar()
for x_tmp in pbar(X_test):
    per_tmp = model.predict(x_tmp.reshape([-1,28*28]))
    label_tmp = np.argmax(per_tmp)
    if label_tmp == np.argmax(Y_test[index1]):
        index2 = index2 +1
    index1 = index1 + 1
    Gini_tmp = 1-np.sum(per_tmp*per_tmp)
#     Gini_tmp = -np.sum(per_tmp*np.log2(per_tmp))
    Gini.append(Gini_tmp)
    predicted_confidence.append(per_tmp)
    
indexs = np.argsort(Gini)
indexs = indexs[::-1]

APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=Y_test, 
                predicted_confidence=np.array(predicted_confidence))
print('APFD:', APFD)
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=Y_test, 
                predicted_confidence=np.array(predicted_confidence))
print('RAUC:', RAUC)

100% |########################################################################|
100% |########################################################################|


APFD: 0.7485526315789472
RAUC: 0.8675030599755202


In [9]:
print('test number:', index1, '\t misclassified number:', index1-index2)

test number: 200 	 misclassified number: 57


In [22]:
# o_i = 0
# for i in range(0, (index1)):
#     x_tmp = X_test[indexs[i]].reshape([-1,28*28])
#     y_tmp = Y_test[indexs[i]]
#     per_tmp = model.predict(x_tmp)
#     if np.argmax(y_tmp)!=np.argmax(per_tmp):
#         o_i = o_i+i
# APFD = 1-o_i/(index1*(index1-index2))+1/(2*index1)
# print(APFD)

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 784)               615440    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_2 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 720,378
Trainable params: 720,378
Non-trainable params: 0
_________________________________________________________________


In [27]:
# use one layer's activation
get_activations = BE.function(inputs=model.inputs[0], outputs=model.layers[-3].output[:,:])
Gini_act = []
pbar = ProgressBar()
for x_tmp in pbar(X_test):    
#     # 使用attention机制
#     x_act = softmax(get_activations(x_tmp.reshape([-1,28*28])))
#     att1 = np.dot(np.transpose(x_act), x_act)
#     att_weight = np.sum(att1, axis = 1).reshape(np.shape(x_act))
#     x_act_ = softmax(x_act*att_weight)
#     Gini_tmp = 1-np.sum(x_act_*x_act_)
#     Gini_tmp = -np.sum(x_act_*np.log2(x_act_))
#     Gini_act.append(Gini_tmp)
    
    # 未使用attention机制
    x_act = softmax(get_activations(x_tmp.reshape([-1,28*28])))
#     Gini_tmp = 1-np.sum(x_act*x_act)
    Gini_tmp = -np.sum(x_act*np.log2(x_act))
    Gini_act.append(Gini_tmp)

# # use all layers' activation
# pbar = ProgressBar()
# get_activations1 = BE.function(inputs=model.inputs[0], outputs=model.layers[-1].output[:,:])
# get_activations2 = BE.function(inputs=model.inputs[0], outputs=model.layers[-2].output[:,:])
# get_activations3 = BE.function(inputs=model.inputs[0], outputs=model.layers[-3].output[:,:])
# Gini_act = []
# for x_tmp in pbar(X_test):
#     x_act1 = softmax(get_activations1(x_tmp.reshape([-1,28*28])))
#     x_act2 = softmax(get_activations2(x_tmp.reshape([-1,28*28])))
#     x_act3 = softmax(get_activations3(x_tmp.reshape([-1,28*28])))
# #     Gini_tmp = 1-np.sum(x_act3*x_act3)
# #     Gini_tmp = 1-(np.sum(x_act1*x_act1)+np.sum(x_act2*x_act2)+np.sum(x_act3*x_act3))/3
#     Gini_tmp = -np.sum(x_act3*np.log2(x_act3))
# #     Gini_tmp = (-np.sum(x_act1*np.log2(x_act1))-np.sum(x_act2*np.log2(x_act2))-np.sum(x_act3*np.log2(x_act3)))/3
#     Gini_act.append(Gini_tmp)
    
indexs = np.argsort(Gini_act)
indexs = indexs[::-1]
APFD = get_APFD(Gini_indexs=indexs, ground_truth_label=Y_test, 
                predicted_confidence=np.array(predicted_confidence))
print('APFD:', APFD)
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=Y_test, 
                predicted_confidence=np.array(predicted_confidence))
print('RAUC:', RAUC)


/public/liujiawei/anaconda3/envs/ZHB_env/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log2
/public/liujiawei/anaconda3/envs/ZHB_env/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in multiply
100% |########################################################################|
100% |########################################################################|


APFD: (0.5150438596491227, 200, 57)
RAUC: 0.5959812321501428


In [19]:
# # use one layer's activation
# get_activations = BE.function(inputs=model.inputs[0], outputs=model.layers[-1].output[:,:])
# Gini_act = []
# pbar = ProgressBar()
# for x_tmp in pbar(X_test):    
#     # 使用attention机制
#     x_act = softmax(get_activations(x_tmp.reshape([-1,28*28])))
#     att1 = np.dot(np.transpose(x_act), x_act)
#     att_weight = np.sum(att1, axis = 1).reshape(np.shape(x_act))
#     x_act_ = softmax(x_act*att_weight)
#     Gini_tmp = 1-np.sum(x_act_*x_act_)
#     Gini_tmp = -np.sum(x_act_*np.log2(x_act_))
#     Gini_act.append(Gini_tmp)
    
#     # 未使用attention机制
#     x_act = softmax(get_activations(x_tmp.reshape([-1,28*28])))
#     Gini_tmp = 1-np.sum(x_act*x_act)
# #     Gini_tmp = -np.sum(x_act*np.log2(x_act))
#     Gini_act.append(Gini_tmp)

# # use all layers' activation
# pbar = ProgressBar()
# get_activations1 = BE.function(inputs=model.inputs[0], outputs=model.layers[-1].output[:,:])
# get_activations2 = BE.function(inputs=model.inputs[0], outputs=model.layers[-2].output[:,:])
# get_activations3 = BE.function(inputs=model.inputs[0], outputs=model.layers[-3].output[:,:])
# Gini_act = []
# for x_tmp in pbar(X_test):
#     x_act1 = softmax(get_activations1(x_tmp.reshape([-1,28*28])))
#     x_act2 = softmax(get_activations2(x_tmp.reshape([-1,28*28])))
#     x_act3 = softmax(get_activations3(x_tmp.reshape([-1,28*28])))
# #     Gini_tmp = 1-np.sum(x_act3*x_act3)
# #     Gini_tmp = 1-(np.sum(x_act1*x_act1)+np.sum(x_act2*x_act2)+np.sum(x_act3*x_act3))/3
#     Gini_tmp = -np.sum(x_act3*np.log2(x_act3))
# #     Gini_tmp = (-np.sum(x_act1*np.log2(x_act1))-np.sum(x_act2*np.log2(x_act2))-np.sum(x_act3*np.log2(x_act3)))/3
#     Gini_act.append(Gini_tmp)
    
# indexs = np.argsort(Gini_act)
# indexs = indexs[::-1]
# o_i = 0
# pbar = ProgressBar()
# for i in pbar(range(0, (index1))):
#     x_tmp = X_test[indexs[i]].reshape([-1,28*28])
#     y_tmp = Y_test[indexs[i]]
#     per_tmp = model.predict(x_tmp)
#     if np.argmax(y_tmp)!=np.argmax(per_tmp):
#         o_i = o_i+i
# APFD = 1-o_i/(index1*(index1-index2))+1/(2*index1)
# print(APFD)

100% |########################################################################|
100% |########################################################################|


APFD: 0.7498684210526315
RAUC: 0.8690330477356181


In [ ]:
APFD 指标基于基尼系数
 使用所有层
0.9357219241773963 使用倒数第三层
0.946923104434907 使用倒数第二层
0.921514556509299 使用倒数第一层，即最后一层（DeepGini）


APFD 指标基于信息熵
0.9608580114449213 使用所有层
0.9608580114449213 使用倒数第三层
0.9326023962804005 使用倒数第二层
0.9212451001430615 使用倒数第一层，即最后一层